In [12]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [13]:
# Player Count
#Display the total number of players - unique values in the SN column
totalplayers = purchase_data["SN"].value_counts()
len(totalplayers) 
totalplayer_df = pd.DataFrame([{"Total Players": len(totalplayers)}])
totalplayer_df

,Total Players
0,576


In [39]:
# Purchase Analysis (total)
# Run basic calculations to obtain number of unique items, average price, number of purchases, and total revenue
unique_items = len(purchase_data["Item ID"].value_counts())
avg_price = purchase_data["Price"].mean()
number_purchases = purchase_data["Purchase ID"].count()
total_revenue = purchase_data["Price"].sum()

# Create a summary data frame to hold the results

purchase_df = pd.DataFrame([{"Number of Unique Items": unique_items,
                "Average Price": avg_price,
                "Number of Purchases": number_purchases,
                "Total Revenue": total_revenue}])

# Give the displayed data cleaner formatting
# Use Map to format all the columns
purchase_df["Average Price"] = purchase_df["Average Price"].map("${:.2f}".format)
purchase_df["Total Revenue"] = purchase_df["Total Revenue"].map("${:,.2f}".format)

# Display the summary data frame
purchase_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [21]:
# Gender Demographics: Percentage and Count of Male Players, Percentage and Count of Female Players
# Percentage and Count of Other / Non-Disclosed

# get just gender and SN, remove duplicates, create series for count and percent; note Age added for use later
gender_SN_df.groupby('SN').count()

gender_SN_df = purchase_data.loc[:, ["Gender", "SN", "Age"]]
gender_SN_unique_df = gender_SN_df.drop_duplicates()
gender_counts = gender_SN_unique_df["Gender"].value_counts()
gender_percents = gender_counts / gender_SN_unique_df["Gender"].count() * 100

# create the gender dataframe
gender_df = pd.DataFrame({
    "Total Count": gender_counts,
    "Percentage": gender_percents
})

# format the % column
gender_df["Percentage"] = gender_df["Percentage"].map("{:.2f}%".format)

# display the dataframe
gender_df


,Total Count,Percentage
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [22]:
# Purchasing Analysis (Gender)
# Run basic calculations to obtain: 
# purchase count, avg. purchase price, total purchase value, and avg. purchase total per person, all by gender
#use groupby "Gender" to get the first three variables, then count, mean, and sum
purchase_count = purchase_data.groupby(["Gender"]).count()["Purchase ID"]
avg_purch_price = purchase_data.groupby(["Gender"]).mean()["Price"]
total_purch_value = purchase_data.groupby(["Gender"]).sum()["Price"]

# to calculate avg_total_purch, use gender_counts from above
avg_total_purch = total_purch_value / gender_counts

# Create a summary data frame to hold the results
summary_df = pd.DataFrame({"Purchase Count": purchase_count,
                          "Average Purchase Price": avg_purch_price,
                          "Total Purchase Value": total_purch_value,
                          "Total Purchase Avg. Per Person": avg_total_purch})

# Optional: give the displayed data cleaner formatting
summary_df["Average Purchase Price"] = summary_df["Average Purchase Price"].map("${:.2f}".format)
summary_df["Total Purchase Value"] = summary_df["Total Purchase Value"].map("${:.2f}".format)
summary_df["Total Purchase Avg. Per Person"] = summary_df["Total Purchase Avg. Per Person"].map("${:.2f}".format)

# Display the summary data frame
summary_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Total Purchase Avg. Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [23]:
# Age Demographics
# Establish bins for ages
bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]

# Create labels for these bins
group_labels = ["<10", "10 to 14", "15 to 19", "20 to 24", "25 to 29", "30 to 34",
                "35 to 39", "40+"]

# Categorize the existing players using the age bins. Hint: use pd.cut()
#Create new column Age Ranges using pd.cut
gender_SN_unique_df["Age Ranges"] = pd.cut(gender_SN_unique_df["Age"], bins, labels=group_labels)

# Calculate the numbers and percentages by age group
counts_by_group = gender_SN_unique_df["Age Ranges"].value_counts()
percent_by_group = counts_by_group / len(totalplayers) * 100

# Create a summary data frame to hold the results
agesummary_df = pd.DataFrame({"Total Count": counts_by_group,
                 "Percentage of Players": percent_by_group})

# Optional: round the percentage column to two decimal points
agesummary_df["Percentage of Players"] = agesummary_df["Percentage of Players"].map("{:.2f}%".format)

# Display Age Demographics Table (after sorting by index)
agesummary_df = agesummary_df.sort_index()
agesummary_df

/Users/marksutton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Total Count,Percentage of Players
<10,17,2.95%
10 to 14,22,3.82%
15 to 19,107,18.58%
20 to 24,258,44.79%
25 to 29,77,13.37%
30 to 34,52,9.03%
35 to 39,31,5.38%
40+,12,2.08%


In [24]:
# Purchasing Analysis (Age)
# Bin the purchase_data data frame by age
bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]

# Create labels for these bins
group_labels = ["<10", "10 to 14", "15 to 19", "20 to 24", "25 to 29", "30 to 34",
                "35 to 39", "40+"]

#use pd.cut to create Age Ranges
purchase_data["Age Ranges"] = pd.cut(purchase_data["Age"], bins, labels=group_labels)

# Run basic calculations to obtain: 
# Purchase Count, Average Purchase Price, Total Purchase Value, Avg Total Purchase per Person
# use groupby "Age Ranges"; this will help create a new df with index "Age Ranges"
grouped_age = purchase_data.groupby(["Age Ranges"]).sum()
purchase_count_agerange = purchase_data.groupby(["Age Ranges"]).count()["Purchase ID"]
avg_purch_price_agerange = purchase_data.groupby(["Age Ranges"]).mean()["Price"]
total_purch_value_agerange = purchase_data.groupby(["Age Ranges"]).sum()["Price"]

#avg_total_purch_agerange - use counts_by_group from above
avg_total_purch_agerange = total_purch_value_agerange / counts_by_group


# Create a summary data frame to hold the results
agesummary2_df = pd.DataFrame({"Purchase Count": purchase_count_agerange,
                          "Average Purchase Price": avg_purch_price_agerange,
                          "Total Purchase Value": total_purch_value_agerange,
                          "Total Purchase Avg. Per Person": avg_total_purch_agerange})

# Optional: give the displayed data cleaner formatting
agesummary2_df["Average Purchase Price"] = agesummary2_df["Average Purchase Price"].map("${:.2f}".format)
agesummary2_df["Total Purchase Value"] = agesummary2_df["Total Purchase Value"].map("${:.2f}".format)
agesummary2_df["Total Purchase Avg. Per Person"] = agesummary2_df["Total Purchase Avg. Per Person"].map("${:.2f}".format)

# Display the summary data frame
agesummary2_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Total Purchase Avg. Per Person
<10,23,$3.35,$77.13,$4.54
10 to 14,28,$2.96,$82.78,$3.76
15 to 19,136,$3.04,$412.89,$3.86
20 to 24,365,$3.05,$1114.06,$4.32
25 to 29,101,$2.90,$293.00,$3.81
30 to 34,73,$2.93,$214.00,$4.12
35 to 39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [25]:
# brandnew_gb_object[["Purchase ID", "Age", "Gender"]].mean()
# Top Spenders
# Find top 5 spenders and create dataframe with:
# Index: SN; Columns: Purchase Count, Average Purchase Price, Total Purchase Value

# First, create new gbdf object by SN
brandnew_gbdf_object = purchase_data.groupby(["SN"])
#brandnew_gbdf_object

#Create new dataframes for each value sought
countperSN = brandnew_gbdf_object[["Purchase ID"]].count()
#countperSN

totalperSN = brandnew_gbdf_object.sum()[["Price"]]
#totalperSN

avgperSN = brandnew_gbdf_object[["Price"]].mean()
#avgperSN

# Since DFs have same index you can join them together to create one DF
merge_df1 = pd.merge(countperSN, avgperSN, on="SN", how="outer")
merge_df2 = pd.merge(merge_df1, totalperSN, on="SN", how="outer")
#merge_df2

#rename columns
renamed_df = merge_df2.rename(columns={"Purchase ID":"Purchase Count", 
                                       "Price_x":"Average Purchase Price",
                                      "Price_y": "Total Purchase Value"})

#sort by total purchase value
sorted_df = renamed_df.sort_values(by=["Total Purchase Value"], ascending=False)

#format the currency columns
sorted_df["Average Purchase Price"] = sorted_df["Average Purchase Price"].map("${:.2f}".format)
sorted_df["Total Purchase Value"] = sorted_df["Total Purchase Value"].map("${:.2f}".format)

#display the final dataframe head -- 5 top values
sorted_df.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [57]:
# Most Popular Items

# Retrieve the Item ID, Item Name, and Item Price columns
# Create new DF with just those three columns
popitem_df = purchase_data[["Item ID", "Item Name", "Price"]]

#Create new series for each value sought
purch_count_df = popitem_df.groupby(["Item ID", "Item Name"]).count()["Price"]
avg_price_df = popitem_df.groupby(["Item ID", "Item Name"]).mean()["Price"]
total_price_df = popitem_df.groupby(["Item ID", "Item Name"]).sum()["Price"]

#create dataframe
renameditem_df = pd.DataFrame({"Count": purch_count_df, 
                                "Average Purchase Price": avg_price_df, 
                                "Total Purchase Value": total_price_df})


#sort by total purchase value
itemsorted_df = renameditem_df.sort_values(by=["Count"], ascending=False)

#format the currency columns
itemsorted_df["Average Purchase Price"] = itemsorted_df["Average Purchase Price"].map("${:.2f}".format)
itemsorted_df["Total Purchase Value"] = itemsorted_df["Total Purchase Value"].map("${:.2f}".format)

itemsorted_df.head()

#type(total_price_df)



,,Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [76]:
# Most Profitable Items
# Sort the above table by total purchase value in descending order
#new_sorted_profit_df = itemsorted_df.sort_values(by=["Total Purchase Value"], ascending=False)
#itemsorted_df = renameditem_df.sort_values(by=["Count"], ascending=False)
#itemsorted_df
new_sorted_profit_df = renameditem_df.sort_values(by=["Total Purchase Value"], ascending=False)


# Optional: give the displayed data cleaner formatting
new_sorted_profit_df["Average Purchase Price"] = new_sorted_profit_df["Average Purchase Price"].map("${:.2f}".format)
new_sorted_profit_df["Total Purchase Value"] = new_sorted_profit_df["Total Purchase Value"].map("${:.2f}".format)

# Display a preview of the data frame
new_sorted_profit_df.head()

,,Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


Three observable trends apparent in the data:

1. In terms of age, we see a normal distribution with the maximum number of customers in the 20 to 24 age bracket, 365, and highest overall spend at $1,117. The numbers get lower at you get to both the younger ages (under 10) and older ages (40 and over). We see a similar pattern in number of players, with the 20-24 having the most players at 258, and fewer players as we move both older and younger from there.

2. The average spend per item stays relatively consistent across all age groups, around $3. The biggest difference in spend by age group is due to the number of users and purchases, not the cost of the items.

3. In terms of gender, males make up the majority of both players (84.03%) and purchases (653). 